In [4]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = [
    "CPA ve prepontin sisternler geniş izlenmektedir. Serebellar folyalar belirgindir. 3. ventrikül ve lateral ventriküller geniş olarak izlenmektedir.   Serebral sulkus ve sisternalar genişlemiştir."
]

tokenizer = Tokenizer(num_words = 100, oov_token="<OOV>")
tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(sentences)

padded = pad_sequences(sequences, maxlen=78)
print("\nWord Index = " , word_index)
print("\nSequences = " , sequences)
print("\nPadded Sequences:")
print(padded)


Word Index =  {'<OOV>': 1, 've': 2, 'geniş': 3, 'izlenmektedir': 4, 'cpa': 5, 'prepontin': 6, 'sisternler': 7, 'serebellar': 8, 'folyalar': 9, 'belirgindir': 10, '3': 11, 'ventrikül': 12, 'lateral': 13, 'ventriküller': 14, 'olarak': 15, 'serebral': 16, 'sulkus': 17, 'sisternalar': 18, 'genişlemiştir': 19}

Sequences =  [[5, 2, 6, 7, 3, 4, 8, 9, 10, 11, 12, 2, 13, 14, 3, 15, 4, 16, 17, 2, 18, 19]]

Padded Sequences:
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  5  2  6  7  3  4  8  9 10 11 12  2 13 14  3 15
   4 16 17  2 18 19]]
